In [1]:
import pandas as pd

## Note that this file is intended to be run with the data in a folder named DATA. Change the data_path below as needed. CHANGING THIS NOTEBOOK'S DATA PATH, SHOULD WORK EXACTLY THE SAME WITH RENT DATA

In [2]:
data_path = "DATA/Metro_zori_uc_sfrcondomfr_sm_month.csv"
Homes_raw = pd.read_csv(data_path)
Homes_raw.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,...,2025-01-31,2025-02-28,2025-03-31,2025-04-30,2025-05-31,2025-06-30,2025-07-31,2025-08-31,2025-09-30,2025-10-31
0,102001,0,United States,country,NaN,1164.805927,1171.136726,1179.812677,1188.754760,1197.580151,...,1907.881861,1917.211710,1929.474463,1938.907827,1946.045220,1950.907629,1953.050444,1953.533991,1952.050075,1948.708033
1,394913,1,"New York, NY",msa,NY,2236.846063,2251.501965,2269.987984,2289.332030,2303.951188,...,3224.984612,3243.454165,3273.056839,3303.707881,3331.069676,3362.757608,3392.266330,3413.333733,3410.432125,3397.903293
2,753899,2,"Los Angeles, CA",msa,CA,1761.871581,1773.071117,1788.065047,1803.355323,1818.396065,...,2871.016877,2893.329888,2917.051681,2915.965206,2919.777391,2924.061198,2926.492700,2927.792645,2927.121443,2924.884110
3,394463,3,"Chicago, IL",msa,IL,1341.017779,1347.973222,1357.967496,1366.921287,1376.295747,...,1975.864028,1992.854981,2010.426073,2031.988128,2051.148618,2069.356824,2080.192420,2083.993205,2082.427779,2076.789127
4,394514,4,"Dallas, TX",msa,TX,1066.712642,1071.569949,1079.056262,1090.463624,1099.855135,...,1665.951348,1673.276725,1682.741809,1692.738538,1695.424062,1696.307700,1693.127850,1689.310560,1684.573082,1678.471598


In [3]:
# Maybe just do top 100 for now
NUM_CITIES = 100
Homes_raw = Homes_raw.head(NUM_CITIES)

# Cleaning the data for Home RENT

### Output is melted_Homes

In [4]:
date_columns = [column for column in Homes_raw.columns if column[:4].isdigit()]

# get first few and last few to verify
print(date_columns[:4])
print(date_columns[-4:])

['2015-01-31', '2015-02-28', '2015-03-31', '2015-04-30']
['2025-07-31', '2025-08-31', '2025-09-30', '2025-10-31']


In [5]:
Homes_raw["RegionName"] = [city[:-4] for city in Homes_raw["RegionName"]]

In [6]:
# need to melt to get to work in px.line
# https://pandas.pydata.org/docs/reference/api/pandas.melt.html
melted_Homes = Homes_raw.melt(
    id_vars = ["RegionName", "StateName"],
    value_vars = date_columns,
    var_name = "Date",
    value_name = "Value"
)
print(melted_Homes.head())
print(melted_Homes.tail())

    RegionName StateName        Date        Value
0    United St       NaN  2015-01-31  1164.805927
1     New York        NY  2015-01-31  2236.846063
2  Los Angeles        CA  2015-01-31  1761.871581
3      Chicago        IL  2015-01-31  1341.017779
4       Dallas        TX  2015-01-31  1066.712642
       RegionName StateName        Date        Value
12995     Augusta        GA  2025-10-31  1479.045173
12996    Palm Bay        FL  2025-10-31  1945.490881
12997     Jackson        MS  2025-10-31  1523.868102
12998  Harrisburg        PA  2025-10-31  1415.610930
12999     Spokane        WA  2025-10-31  1516.291808


In [7]:
# turn to datetime so can do cool stuff with it, right now is string
# ONLY RUN THIS CELL ONCE
melted_Homes["Date"] = pd.to_datetime(melted_Homes["Date"])

# Creation of Interactive Linechart

In [8]:
import plotly.express as px

In [9]:
# https://plotly.com/python/line-charts/

max_y = melted_Homes['Value'].max() * 1.05
min_y = 0

fig = px.line(melted_Homes,
             x = 'Date',
             y = 'Value',
             color = 'StateName', # maybe change this later but cool to group together states
             hover_name = 'RegionName', # change hover to have cooler stuff later
             line_group = 'RegionName',
             # color_discrete_sequence = px.colors.qualitative.Set1,
             labels = {
                 'RegionName':'City',
                 'StateName':'State',
                 'Date':'Date',
                 'Value':'Average Rent'
             },
             width = 1400,
             height = 920,
             range_y = [min_y, max_y],
             # hover_data = {'Importance':False,
             #              'Value':True,
             #              'Question':False,
             #              'ShortQ':False}
)

fig.update_layout(
    title = {'text':"City Rents Increase Over Time and Spike in the Summer", 'xanchor':'left', 'font':{'size':24,'weight':'bold'}} # go back and change title this is just for now
)

fig.write_image("AllStatesRentTimeSeries.png")

fig.write_html("AllStatesRentTimeSeries.html")

fig.show()
### INITAL VERSION not great way too many lines and colors, could gray them out but then state information is lost. So IDK what's best to do we'll go with this for now
# because interactive = can CLICK certain states
             

C:\Users\poppy\PythonStuff\DataVis\Vis_venv\Lib\site-packages\choreographer\utils\_tmpfile.py:137: TmpDirWarning:

The temporary directory could not be deleted, execution will continue. errors: [(WindowsPath('C:/Users/poppy/AppData/Local/Temp/tmp7ontzknn/Default/Asset Store/assets.db/000003.log'), PermissionError(13, 'Access is denied')), (WindowsPath('C:/Users/poppy/AppData/Local/Temp/tmp7ontzknn/Default/Asset Store/assets.db/CURRENT'), PermissionError(13, 'Access is denied')), (WindowsPath('C:/Users/poppy/AppData/Local/Temp/tmp7ontzknn/Default/Asset Store/assets.db/LOCK'), PermissionError(13, 'Access is denied')), (WindowsPath('C:/Users/poppy/AppData/Local/Temp/tmp7ontzknn/Default/Asset Store/assets.db/LOG'), PermissionError(13, 'Access is denied')), (WindowsPath('C:/Users/poppy/AppData/Local/Temp/tmp7ontzknn/Default/Asset Store/assets.db/MANIFEST-000001'), PermissionError(13, 'Access is denied')), (WindowsPath('C:/Users/poppy/AppData/Local/Temp/tmp7ontzknn/Default/Asset Store/assets